**Лабараторная работа 1 Создание ML-пайплайна, Git и FastAPI в Google Colab**

In [65]:
!pip install xgboost scikit-learn joblib pandas numpy fastapi uvicorn pyngrok

In [66]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [69]:
import pandas as pd
from sklearn.model_selection import train_test_split
file_path = '/content/drive/MyDrive/Laptop_price.csv'
df = pd.read_csv(file_path)
X = df.drop(columns=['Price'])
y = df['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [70]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
import joblib

num_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_features = X.select_dtypes(include=['object']).columns.tolist()

num_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])


cat_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])


preprocessor = ColumnTransformer([
    ('num', num_transformer, num_features),
    ('cat', cat_transformer, cat_features)
])


pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5))
])

pipeline.fit(X_train, y_train)

joblib.dump(pipeline, '/content/drive/MyDrive/laptop_price_model.pkl')

['/content/drive/MyDrive/laptop_price_model.pkl']

In [6]:
!git init

Reinitialized existing Git repository in /content/.git/


In [8]:
#Добавим коммит
!git add .
!git commit -m "Добавлен ML-пайплайн"

^C
Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@b8c215f47f64.(none)')


In [14]:
#Авторизуемся
!git config --global user.email "morrainyt@gmail.com"
!git config --global user.name "Dunin2302"

In [15]:
!git add .
!git commit -m "Добавлен ML-пайплайн"

[master (root-commit) 958757f] Добавлен ML-пайплайн
 26 files changed, 56188 insertions(+)
 create mode 100644 .config/.last_opt_in_prompt.yaml
 create mode 100644 .config/.last_survey_prompt.yaml
 create mode 100644 .config/.last_update_check.json
 create mode 100644 .config/active_config
 create mode 100644 .config/config_sentinel
 create mode 100644 .config/configurations/config_default
 create mode 100644 .config/default_configs.db
 create mode 100644 .config/gce
 create mode 100644 .config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db
 create mode 100644 .config/logs/2025.05.09/13.40.57.224020.log
 create mode 100644 .config/logs/2025.05.09/13.41.18.092482.log
 create mode 100644 .config/logs/2025.05.09/13.41.26.495445.log
 create mode 100644 .config/logs/2025.05.09/13.41.27.718573.log
 create mode 100644 .config/logs/2025.05.09/13.41.36.263788.log
 create mode 100644 .config/logs/2025.05.09/13.41.36.930373.log
 create mode 100644 drive/MyDrive/Colab Notebooks/111

In [16]:
!git remote add origin https://github.com/Dunin2302/Labagit # Добавляем удалённый репозиторий GitHub с псевдонимом "origin"

In [40]:
!ssh-keygen -t ed25519 -C "morrainyt@gmail.com" # Генерируем новый SSH-ключ

Generating public/private ed25519 key pair.
Enter file in which to save the key (/root/.ssh/id_ed25519): 
/root/.ssh/id_ed25519 already exists.
Overwrite (y/n)? y
Enter passphrase (empty for no passphrase): 
Enter same passphrase again: 
Your identification has been saved in /root/.ssh/id_ed25519
Your public key has been saved in /root/.ssh/id_ed25519.pub
The key fingerprint is:
SHA256:XJu0hzLZtSk+aUtSJYPsujLE00L5lUONf24plL8x0sI morrainyt@gmail.com
The key's randomart image is:
+--[ED25519 256]--+
|         o       |
|       .o..      |
|     . .oo=.o    |
|    o  o+=+X.o   |
|   o o .S+O=+.   |
|    = o. =E+O    |
|   . o. . *= +   |
|    o  . + o.    |
|     o.   .      |
+----[SHA256]-----+


In [41]:
!cat ~/.ssh/id_ed25519.pub #Копируем ключ

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIE9oCjfohN2baT7f2U2PZuLVc19iQa20qTLkNG/0A2uG morrainyt@gmail.com


In [43]:
!ssh-keyscan github.com >> /root/.ssh/known_hosts # Добавляем публичный SSH-ключ GitHub в список известных хостов для безопасного соединения

# github.com:22 SSH-2.0-0264bb16
# github.com:22 SSH-2.0-0264bb16
# github.com:22 SSH-2.0-0264bb16
# github.com:22 SSH-2.0-0264bb16
# github.com:22 SSH-2.0-0264bb16


In [44]:
!ssh -T git@github.com # Тестируем соединение с GitHub по SSH

Hi Dunin2302! You've successfully authenticated, but GitHub does not provide shell access.


In [45]:
!git remote -v # Показываем список удалённых репозиториев с указанием их URL

origin	https://github.com/Dunin2302/Labagit (fetch)
origin	https://github.com/Dunin2302/Labagit (push)


In [46]:
!git add .
!git commit -m "Добавлен ML-пайплайн"

[master fa9278c] Добавлен ML-пайплайн
 3 files changed, 9 insertions(+), 1 deletion(-)
 rewrite "drive/MyDrive/Colab Notebooks/\320\224\321\203\320\275\320\270\320\275_Lab1.ipynb" (88%)
 create mode 100644 key
 create mode 100644 key.pub


In [47]:
!git config --global credential.helper wincred # Включаем хранение учетных данных Git в менеджере учетных данных Windows

In [55]:
!git remote set-url origin git@github.com:Dunin2302/Labagit.git # Изменяем адрес удалённого репозитория на SSH-версию для более удобной и защищённой работы с GitHub

In [59]:
!git remote add origin git@github.com:Dunin2302/Labagit.git
!git branch -M main
!git push -u origin main

error: remote origin already exists.
Branch 'main' set up to track remote branch 'main' from 'origin'.
Everything up-to-date


**Развертывание **FastAPI**

In [60]:
%%writefile app.py
# Создание REST API с помощью FastAPI для предсказания цен ноутбуков на основе обученной модели
from fastapi import FastAPI, File, UploadFile
import pandas as pd
import joblib
from io import BytesIO

app = FastAPI()

# Загружаем обученную модель из Google Диска
model_path = "/content/drive/MyDrive/laptop_price_model.pkl"
model = joblib.load(model_path)

# Роут для POST-запросов, принимающих CSV-файлы с признаками ноутбука
@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
# Читаем входящий файл в байтовый поток
content = await file.read()

# Парсим полученные данные в DataFrame
df = pd.read_csv(BytesIO(content))

# Прогоняем данные через модель и получаем прогнозы
predictions = model.predict(df)

# Возвращаем спрогнозированные цены в виде списка
return {"predictions": predictions.tolist()}

Writing app.py


In [61]:
# Устанавливаем библиотеку python-multipart для поддержки обработки multipart-загрузки файлов в приложениях FastAPI
!pip install python-multipart

In [62]:
!ngrok config add-authtoken 2x0L4CxzTB4twUdO5eh8ghW8vKx_4twuc8bixA6YYvmERb2Ux # Добавляем токен аутентификации ngrok для возможности создавать туннели и прокси-сервисы

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [63]:
!nohup uvicorn app:app --host 0.0.0.0 --port 8000 --reload > fastapi.log 2>&1 & # Запускаем сервер FastAPI в фоновом режиме с авто-перезагрузкой и ведением журнала активности

In [64]:
from pyngrok import ngrok
# Подключаемся к службе ngrok для создания временного публичного URL, обеспечивающего доступ к нашему локальному серверу FastAPI
public_url = ngrok.connect(8000)
print("API доступно по адресу:", public_url)

API доступно по адресу: NgrokTunnel: "https://bd54-34-91-5-2.ngrok-free.app" -> "http://localhost:8000"


In [ ]:
import pandas as pd

pd.set_option('display.max_rows', None)      # Отображаем ВСЕ строки
pd.set_option('display.max_columns', None)   # Отображаем ВСЕ столбцы
pd.set_option('display.width', None)         # Без ограничения ширины экрана
pd.set_option('display.expand_frame_repr', False)  # Запрещаем перенос длинных строк

# Наш DataFrame
df = pd.read_csv('/content/drive/MyDrive/Laptop_price_secure.csv')

# Вывод всей таблицы
df

,Brand,Processor_Speed,RAM_Size,Storage_Capacity,Screen_Size,Weight,Price,Price_Hashed,Price_Encrypted,RAM_Size_Encrypted
0,Asus,3.830296,16,512,11.185147,2.641094,17395.093065,e33562234b73b4ecad16901885fd503f2c4c20996e22d9...,gAAAAABoD7xRkOEwx0bxuxSDs-7LW6WL7fDTSbmM7ilcKH...,gAAAAABoD7xRC8XDpZEfELdx9OGaSw_ZYp0JXSjrevW1wu...
1,Acer,2.912833,4,1000,11.311372,3.260012,31607.605919,80fe437d4a9c57c4038198b7cf32f93b9f957bb705b8e1...,gAAAAABoD7xRxOeQlcP5h3x3Dw-kkJMzgkvjjLWncDHru9...,gAAAAABoD7xROVxw6qoJQK0Acr3g9fvmTCR4Z6OJ8Xx0eC...
2,Lenovo,3.241627,4,256,11.853023,2.029061,9291.023542,6254a5a18d60d796eb464283ef38f9417e12885a07742e...,gAAAAABoD7xRUMJ_f2aaAmFzmUvvodc-9_b-X3V4N7bvWl...,gAAAAABoD7xRqtwYa618qIzdALzKH4H5MtLqn2GOcid1uC...
3,Acer,3.806248,16,512,12.280360,4.573865,17436.728334,1f2910923c33a53b30bc7e6eb731307eb2e2366b3811b6...,gAAAAABoD7xRvEUAxrJQF0v0ajc6084-dmZnRd0WmPEiev...,gAAAAABoD7xR2IEjCxYVy1QqGNRZwWhrLgAS7K-yi2pLyB...
4,Acer,3.268097,32,1000,14.990877,4.193472,32917.990718,f90738a55c90c1e31a4fb6d35f92306ff03a87001a9408...,gAAAAABoD7xRVi9lyfUUctuHOnQ9nrURVtwzgejybFx_aj...,gAAAAABoD7xRbY7CVQNng8pgmDhQhtDPfWH5n8NuuBTe1z...
5,HP,1.881348,16,256,11.943958,4.840268,9543.719980,152ff98902e83cca39225afe41781193058d58e2821954...,gAAAAABoD7xRY3n62yXY7gJwnkItLjXgb5nZLfeoKJEtlw...,gAAAAABoD7xRAEeC0bTsvic65aQBdZxkQDxs7QmkThdc7F...
6,Lenovo,2.940721,32,256,11.713246,4.344846,10390.137319,90d3e3d05c2cf26f052a507df2d7af7733f4784154877c...,gAAAAABoD7xRQRY4MqKwveXwWXBnQe8iIJb9RH2MlyUieM...,gAAAAABoD7xR0kwBcKTbjySWOSh9y_jDiHc6o4D499FGPH...
7,Lenovo,3.016788,4,256,15.588315,2.941120,9214.484506,981559f507d2055d3f2ff381b07fab0a3af19f070280bb...,gAAAAABoD7xRywg4BRMwXX-NyfWQSfRoPLVnm4Z1ZAZ4zs...,gAAAAABoD7xRSS-walzx_feDHfW9OD-TlmkTJjlZECC1od...
8,Lenovo,2.560327,4,512,15.026332,2.743620,16372.175284,9d1993a7420329974c5b60f45bcc22a92da4598d58acb1...,gAAAAABoD7xR2Xi25VY-bNoa7wt0mXCaSR0mdEXhcTMTM3...,gAAAAABoD7xRCKr_17133nM_Sep0G5OnCr0kAYSyUWuDB5...
9,Acer,3.341111,32,512,13.611945,4.093149,18501.832733,1ab75c9a76df5af1a9800bc1c2394bff5334234916f104...,gAAAAABoD7xRrUxQ_qVZ43liACiX6cyJQwgf1vLr8F2Q_y...,gAAAAABoD7xRgP59SzgNXbNC02wTbWQjIUatuww1Y3h2zL...
